## Import
~

In [123]:
import pandas as pd
from selenium import webdriver
import time
import datetime
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Browser Automation With Selenium

In [124]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
#driver.maximize_window()
driver.get('https://www.imdb.com/')

## dropdown
dropdown=driver.find_element(By.CLASS_NAME,'ipc-icon--arrow-drop-down')
dropdown.click()

time.sleep(1)

## advanced search from dropdown
element = driver.find_element(By.LINK_TEXT,'Advanced Search')
element.click()

##click avanced title search 
adv_title_search = driver.find_element(By.LINK_TEXT,'Advanced Title Search')
adv_title_search.click()

##select tv movie title search 
sel_tv_movie = driver.find_element(By.ID,'title_type-2')
sel_tv_movie.click()


##select fetaure film
sel_fet_film = driver.find_element(By.ID,'title_type-1')
sel_fet_film.click()

# enter minimum date
min_date = driver.find_element(By.NAME, 'release_date-min')
min_date.click()
min_date.send_keys("1990")

# enter maximum date
max_date = driver.find_element(By.NAME, 'release_date-max')
max_date.click()
max_date.send_keys(str(datetime.datetime.now().year))

#select minimum rating
rating_min = driver.find_element(By.NAME, 'user_rating-min')
rating_min.click()
select_from_dropdown_min = Select(rating_min)
select_from_dropdown_min.select_by_visible_text('1.0')

#select maximum rating
rating_max = driver.find_element(By.NAME, 'user_rating-max')
rating_max.click()
select_from_dropdown_max = Select(rating_max)
select_from_dropdown_max.select_by_visible_text('10')

#select oscar nominated
oscar_nominated = driver.find_element(By.ID, 'groups-7')
oscar_nominated.click()

#select colored
color_movies = driver.find_element(By.ID, 'colors-1')
color_movies.click()

#select from dropdown english
language = driver.find_element(By.NAME, 'languages')
select_from_dropdown_lang = Select(language)
select_from_dropdown_lang.select_by_visible_text('English')

#select from dropdown pages per resulkt
pages = driver.find_element(By.ID, 'search-count')
select_from_dropdown_pages = Select(pages)
select_from_dropdown_pages.select_by_value('250')

##submit
submit = driver.find_element(By.XPATH, '(//button[@type="submit"])[2]')
submit.click()

#current
current_url = driver.current_url

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\lenovo\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


# Data Extraction By beautiful Soup

In [125]:
# get request
response = requests.get(current_url)

#get statuscode
response.status_code


200

In [126]:
# soup object
soup = BeautifulSoup(response.content, 'html.parser')
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film/TV Movie,
Released between 1990-01-01 and 2021-12-31,
User Rating between 1 and 10,
Oscar-Nominated,
Color,
English
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
<

In [127]:
# result items (starting point)
list_items = soup.findAll('div', {'class': 'lister-item'})
len(list_items)

250

In [128]:
list_items

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt2379713"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt2379713/"> <img alt="Spectre" class="loadlate" data-tconst="tt2379713" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BOWQ1MDE1NzgtNTQ4OC00ZjliLTllZDAtN2IyOTVmMTc5YjUxXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt2379713/">Spectre</a>
 <span class="lister-item-year text-muted unbold">(I) (2015)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">PG-13</span>
 <span class="ghost">|</span>
 <span class="runtime">148 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Action, Ad

## Data We need to extract

- movie title
- year
- duration
- genre
- rating

### Individual Element Extraction

In [129]:
# movie title
title =list_items[0].find('h3').find('a').get_text()
title

'Spectre'

In [130]:
# movie year
## another method :list_items[0].find('h3').find_all('span')[1].get_text()
year =list_items[0].find('h3').find('span',{'class':'lister-item-year'}).get_text()
year = str(''.join(filter(str.isdigit, year)))
year

'2015'

In [131]:
#duration
duration = list_items[0].find('span',{'class':'runtime'}).get_text()
duration

'148 min'

In [132]:
#genre
genre = list_items[0].find('span',{'class':'genre'}).get_text().strip()
genre

'Action, Adventure, Thriller'

In [133]:
#ratings
rating = list_items[0].find('div',{'class':'ratings-bar'}).find('div',{'class':'ratings-imdb-rating'}).get_text().strip()
rating

'6.8'

### All Elements Extraction

In [134]:
# list comprehension to get all elements
title =[result.find('h3').find('a').get_text() for result in list_items]
year =[str(''.join(filter(str.isdigit, result.find('h3').find('span',{'class':'lister-item-year'}).get_text()))) for result in list_items]
duration = [result.find('span',{'class':'runtime'}).get_text() for result in list_items]
genre = [result.find('span',{'class':'genre'}).get_text().strip() for result in list_items]
rating = [result.find('div',{'class':'ratings-bar'}).find('div',{'class':'ratings-imdb-rating'}).get_text().strip() for result in list_items]




In [135]:
# create dataframe
imdb_df = pd.DataFrame({'Movie title': title, 'Year': year, 'Duration': duration, 'Genre': genre, 'Rating': rating})
imdb_df

,Movie title,Year,Duration,Genre,Rating
0,Spectre,2015,148 min,"Action, Adventure, Thriller",6.8
1,Skyfall,2012,143 min,"Action, Adventure, Thriller",7.8
2,Knives Out,2019,130 min,"Comedy, Crime, Drama",7.9
3,Once Upon a Time... in Hollywood,2019,161 min,"Comedy, Drama",7.6
4,Blade Runner 2049,2017,164 min,"Action, Drama, Mystery",8.0
...,...,...,...,...,...
245,"Girl, Interrupted",1999,127 min,"Biography, Drama",7.3
246,Silver Linings Playbook,2012,122 min,"Comedy, Drama, Romance",7.7
247,The Mask,1994,101 min,"Action, Comedy, Crime",6.9
248,Judas and the Black Messiah,2021,126 min,"Biography, Drama, History",7.5


In [137]:
# output in excel
imdb_df.to_excel('imdb_multiple_pages.xlsx', index=False)